In [392]:
import json

# Used to display points and shapes on a map
from ipyleaflet import GeoJSON, Map

# Main package
from shape2geosparql import convert

# Used to run geospatial queries on Fuseki
from SPARQLWrapper import GET, JSON, POST, SPARQLWrapper

### Converting the shapefile to triples

In [393]:
# Convert a shapefile
converter = convert('data/trento_stations/stazioni.shp')

In [394]:
# How do the triples look like?

print(converter.write().decode())

<http://www.example.org/shape2geosparql/stazioni/data/9_geom> <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://www.opengis.net/ont/sf#Point> .
<http://www.example.org/shape2geosparql/stazioni/data/5_geom> <http://www.opengis.net/ont/geosparql#asWKT> "POINT (11.107593 46.12596)"^^<http://www.opengis.net/ont/geosparql#wktLiteral> .
<http://www.example.org/shape2geosparql/stazioni/data/3_geom> <http://www.w3.org/2003/01/geo/wgs84_pos#long> "11.104557999999999"^^<http://www.w3.org/2001/XMLSchema#double> .
<http://www.example.org/shape2geosparql/stazioni/data/1_geom> <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://www.opengis.net/ont/sf#Geometry> .
<http://www.example.org/shape2geosparql/stazioni/data/0_geom> <http://github.com/nvitucci/shape2geosparql/ontology#asGeoJson> "{ \"type\": \"Point\", \"coordinates\": [ 11.120056, 46.0726 ] }" .
<http://www.example.org/shape2geosparql/stazioni/data/1> <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://www.opengis.net/ont

In [409]:
# A different serialization format can be used ('nt' being the default)

print(converter.write(outformat='turtle').decode())

@prefix ns1: <http://www.w3.org/2003/01/geo/wgs84_pos#> .
@prefix ns2: <http://github.com/nvitucci/shape2geosparql/ontology#> .
@prefix ns3: <http://www.example.org/shape2geosparql/stazioni/ontology/> .
@prefix ns4: <http://www.opengis.net/ont/geosparql#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xml: <http://www.w3.org/XML/1998/namespace> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<http://www.example.org/shape2geosparql/stazioni/data/0> a <http://www.opengis.net/ont/sf#Feature> ;
    ns3:nome "FS - Trento" ;
    ns3:tratta "FFSS" ;
    ns4:hasGeometry <http://www.example.org/shape2geosparql/stazioni/data/0_geom> .

<http://www.example.org/shape2geosparql/stazioni/data/1> a <http://www.opengis.net/ont/sf#Feature> ;
    ns3:nome "FTM - Trento" ;
    ns3:tratta "FTM" ;
    ns4:hasGeometry <http://www.example.org/shape2geosparql/stazioni/data/1_geom> .

<http://www.example.org/shape2geosparql/s

In [410]:
# Get the actual graph to run some SPARQL queries

graph = converter.graph

# The GeoSPARQL ontology does not define a property to serialize geometries as GeoJSON,
# so we use the one defined in shape2geosparql; we also use the automatically-generated
# ontology that describes the features
geojson = graph.query('''
    PREFIX geosparql: <http://www.opengis.net/ont/geosparql#>
    PREFIX s2g: <http://github.com/nvitucci/shape2geosparql/ontology#>
    PREFIX ont: <http://www.example.org/shape2geosparql/stazioni/ontology/>
    
    SELECT ?feature ?geojson
    WHERE {
        ?feature ont:nome "FS - Trento"; 
                 geosparql:hasGeometry ?geom .
        ?geom s2g:asGeoJson ?geojson
    }
''')

print(geojson.bindings)

[{rdflib.term.Variable('feature'): rdflib.term.URIRef('http://www.example.org/shape2geosparql/stazioni/data/0'), rdflib.term.Variable('geojson'): rdflib.term.Literal('{ "type": "Point", "coordinates": [ 11.120056, 46.0726 ] }')}]


In [411]:
# Get the GeoJSON content
geojson_value = str(geojson.bindings[0]['geojson'])

print(geojson_value)

{ "type": "Point", "coordinates": [ 11.120056, 46.0726 ] }


### Displaying the points on a map

In [412]:
# Create a map and center it on Trento, Italy
m = Map(center=(46.07, 11.15), zoom=13)

In [413]:
# Create a GeoJSON layer from our GeoJSON string
geojson_layer = GeoJSON(data=json.loads(geojson_value))

# Add the layer to the map
m.add_layer(geojson_layer)

In [414]:
# Display the map
m

Map(center=[46.07, 11.15], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_o…

In [415]:
# Add all the points instead of just one

geojson = graph.query('''
    PREFIX geosparql: <http://www.opengis.net/ont/geosparql#>
    PREFIX s2g: <http://github.com/nvitucci/shape2geosparql/ontology#>
    PREFIX ont: <http://www.example.org/shape2geosparql/stazioni/ontology/>
    
    SELECT ?geom ?geojson
    WHERE {
        ?geom s2g:asGeoJson ?geojson
    }
''')

geojson_values = map(lambda el: json.loads(str(el['geojson'])), geojson.bindings)

In [416]:
# Display all the points on the map, one point per layer

m = Map(center=(46.07, 11.15), zoom=12)

for geojson_value in geojson_values:
    geo_json = GeoJSON(data=geojson_value)
    m.add_layer(geo_json)

m

Map(center=[46.07, 11.15], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_o…

### Displaying polygons on a map

In [417]:
converter = convert('data/trento_parking/zone_parcheggio.shp')
graph = converter.graph

geojson = graph.query('''
    PREFIX geosparql: <http://www.opengis.net/ont/geosparql#>
    PREFIX s2g: <http://github.com/nvitucci/shape2geosparql/ontology#>
    PREFIX ont: <http://www.example.org/shape2geosparql/stazioni/ontology/>
    
    SELECT ?geom ?geojson
    WHERE {
        ?geom s2g:asGeoJson ?geojson
    }
''')

geojson_values = map(lambda el: json.loads(str(el['geojson'])), geojson.bindings)

In [418]:
m = Map(center=(46.07, 11.15), zoom=13)

# Define a style for the polygons
style = {'color': 'blue', 'weight': 1.5, 'dashArray': '5', 'fillOpacity': 0.3}

for geojson_value in geojson_values:
    geo_json = GeoJSON(data=geojson_value, style=style)
    m.add_layer(geo_json)

m

Map(center=[46.07, 11.15], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_o…

### Running geospatial queries

In order to run the queries in the following section, Fuseki GeoSPARQL server must be running (see the README file)

In [419]:
# Converting the first dataset again, for convenience
converter = convert('data/trento_stations/stazioni.shp')

In [420]:
# Both the query endpoint and the update endpoint are needed, since the
# GeoSPARQL dataset has to be loaded on the server
endpoint = SPARQLWrapper('http://localhost:3030/ds/sparql', updateEndpoint='http://localhost:3030/ds/update')

In [421]:
# Insert the data with a standard "INSERT DATA" query; to do this, the data have to be:
# - serialized in the 'nt' (N-Triples) format;
# - reasonably small.
# If the data are large, either insert them in batches or serialize them in a file to be uploaded.
endpoint.setMethod(POST)
endpoint.setQuery('INSERT DATA {%s}' % converter.write(outformat='nt').decode())

# Nothing needs to be returned
endpoint.query()

In [422]:
endpoint.setMethod(GET)
endpoint.setReturnFormat(JSON)

# Besides the custom and the GeoSPARQL namespaces, two more namespaces are used:
# - the "units" namespace, which defines the units (e.g. kilometers, meters, miles, etc.)
# for the geospatial queries (see https://jena.apache.org/documentation/geosparql/index#units-uri)
# - the Apache Jena geospatial filter functions namespace, to be used in FILTERs
# (see https://jena.apache.org/documentation/geosparql/index#filter-functions)
#
# Other namespaces are available for more operations 
# (https://jena.apache.org/documentation/geosparql/index#apache-jena-spatial-functionswgs84-geo-predicates).
#
# In this query, we look for features within 1 km from the specified point having
# latitude = 46.054385 and longitude = 11.13544.

endpoint.setQuery('''
    PREFIX geo: <http://www.opengis.net/ont/geosparql#>
    PREFIX units: <http://www.opengis.net/def/uom/OGC/1.0/>
    PREFIX spatialF: <http://jena.apache.org/function/spatial#>
    PREFIX ont: <http://www.example.org/shape2geosparql/stazioni/ontology/>
    
    SELECT ?feature ?name ?geom 
    WHERE {
        ?feature ont:nome ?name;
                 geo:hasGeometry/geo:asWKT ?geom .
        BIND (spatialF:convertLatLon(46.054385, 11.13544) AS ?point)
        FILTER (spatialF:nearby(?geom, ?point, 1, units:kilometer))}
''')

res = endpoint.query()

In [408]:
# See the results in JSON format
print(json.dumps(res.convert(), indent=2))

{
  "head": {
    "vars": [
      "feature",
      "name",
      "geom"
    ]
  },
  "results": {
    "bindings": [
      {
        "feature": {
          "type": "uri",
          "value": "http://www.example.org/shape2geosparql/stazioni/data/6"
        },
        "name": {
          "type": "literal",
          "value": "VALS - S. Chiara"
        },
        "geom": {
          "type": "literal",
          "datatype": "http://www.opengis.net/ont/geosparql#wktLiteral",
          "value": "POINT (11.13544 46.054385)"
        }
      },
      {
        "feature": {
          "type": "uri",
          "value": "http://www.example.org/shape2geosparql/stazioni/data/7"
        },
        "name": {
          "type": "literal",
          "value": "VALS - S. Bartolomeo"
        },
        "geom": {
          "type": "literal",
          "datatype": "http://www.opengis.net/ont/geosparql#wktLiteral",
          "value": "POINT (11.135147 46.04746)"
        }
      }
    ]
  }
}
